In [1]:
import numpy as np
from sklearn.metrics import pairwise_distances
from gtda.homology import VietorisRipsPersistence

In [2]:
def phi(curvature,u_vect):
    if curvature > 0:
        r = (2/np.sqrt(curvature)) * np.arcsin(np.sqrt(u_vect) * np.sin(np.sqrt(curvature)/2))
    if curvature == 0:
        r = np.sqrt(u_vect)
    if curvature < 0:
        r = (2/np.sqrt(-curvature)) * np.arcsinh(np.sqrt(u_vect) * np.sinh(np.sqrt(-curvature)/2))
    return r    
        

def sample_uniformly(curvature, n_points):
    theta = 2 * np.pi * np.random.random_sample((n_points,))
    r = phi(curvature, np.random.random_sample((n_points,)))
    return np.stack((r,theta), axis = -1)
        
def geodesic_distance(curvature, x1 , x2):
    
    if curvature > 0:
        R = 1/np.sqrt(curvature)
        v1 = np.array([R * np.sin(x1[0]/R) * np.cos(x1[1]), 
                       R * np.sin(x1[0]/R) * np.sin(x1[1]),
                      R * np.cos(x1[0]/R)])
        
        v2 = np.array([R * np.sin(x2[0]/R) * np.cos(x2[1]), 
                       R * np.sin(x2[0]/R) * np.sin(x2[1]),
                      R * np.cos(x2[0]/R)])

        
        dist = R * np.arctan2(np.linalg.norm(np.cross(v1,v2)), (v1*v2).sum())
    
    if curvature == 0:
        v1 = np.array([x1[0]*np.cos(x1[1]), x1[0]*np.sin(x1[1])])
        v2 = np.array([x2[0]*np.cos(x2[1]), x2[0]*np.sin(x2[1])])
        dist = np.linalg.norm( (v1 - v2) )  
    
    if curvature < 0:
        R = 1/np.sqrt(-curvature)
        z = np.array([ np.tanh(x1[0]/(2 * R)) * np.cos(x1[1]),
                       np.tanh(x1[0]/(2 * R)) * np.sin(x1[1])])
        w = np.array([np.tanh(x2[0]/(2 * R)) * np.cos(x2[1]),
                       np.tanh(x2[0]/(2 * R)) * np.sin(x2[1])])
        temp = np.linalg.norm([(z*w).sum() - 1, np.linalg.det([z,w]) + 1])
        dist = 2 * R * np.arctanh(np.linalg.norm(z - w)/temp) 
        
    return dist

def distance_matrix(curvature,n_points):
    metric = lambda x1, x2 : geodesic_distance(curvature, x1 , x2)
    samples = sample_uniformly(curvature, n_points)
    return pairwise_distances(samples, metric = metric, n_jobs=12)

In [3]:
def get_diagrams(curvatures, n_points, hodim):  
    """ This functions outputs the persistence diagrams in homological dimensions given by hodim, 
    for a list datasets obained by uniformly sampling n_points for each elements in curvatures
    
    Args:
        curvatures (list of float):
            List of values of the curvatures of surfaces to sample from
        n_points (int):
            Number of points to be sampled on each surface
        hodim (list of int):
            Dimensions in which to compute persistenc diagrams
    """
    
    l = []
    VR = VietorisRipsPersistence(homology_dimensions=hodim, metric = 'precomputed')
    for curvature in curvatures:
        matrix = distance_matrix(curvature,n_points)
        l.append(matrix)
    diagrams = VR.fit_transform(l)
    return diagrams

In [15]:
n_points = 300
curvatures =  np.random.uniform(-2, 2, 20)
hodim = (0,1)

In [5]:
get_diagrams(curvatures, n_points, hodim);

In [17]:
from joblib import Parallel, delayed



def process(i):
    return distance_matrix(curvatures[i], n_points)
    
pds = Parallel(n_jobs=10)(delayed(process)(i) for i in range(len(curvatures)))

np.stack(pds, axis=0).shape

(20, 300, 300)

In [7]:
d_mat[0] = np.array([1, 1])

NameError: name 'd_mat' is not defined

In [ ]:
process(1)

In [ ]:
d_mat

[None,
 array([[0.        , 1.18108998, 0.83212347, ..., 1.04505694, 1.25353806,
         0.30320705],
        [1.18108998, 0.        , 0.83228197, ..., 0.25322695, 0.30109488,
         0.89609082],
        [0.83212347, 0.83228197, 0.        , ..., 0.58205325, 1.08478027,
         0.58041883],
        ...,
        [1.04505694, 0.25322695, 0.58205325, ..., 0.        , 0.53442013,
         0.74437678],
        [1.25353806, 0.30109488, 1.08478027, ..., 0.53442013, 0.        ,
         1.00606503],
        [0.30320705, 0.89609082, 0.58041883, ..., 0.74437678, 1.00606503,
         0.        ]]),
 None]